In [449]:
# -*- coding: utf-8 -*-
"""
compute sentence similarity usning Doc2Vec
Xiaofeng Liao,
13 May, 2018
This script is based on this post:
https://medium.com/@klintcho/doc2vec-tutorial-using-gensim-ab3ac03d3a1
"""
import gensim
from gensim.models.doc2vec import LabeledSentence
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


LabeledSentence = gensim.models.doc2vec.LabeledSentence
#sentence = LabeledSentence(words=[u'some', u'words', u'here'], tags=[u'SENT_1'])

from os import listdir
from os.path import isfile, join

class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
          #print(idx)
          #print(doc)
          words=doc.split()
          tags = [self.labels_list[idx]]
          #print('in LabeledLineSentence words')
          #print(words)
          #print('in LabeledLineSentence tag')
          #print(tags)
          yield LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])


#now create a list that contains the name of all the text file in your data #folder
docLabels = []
docLabels = [f for f in listdir("inputfile/") if f.endswith('.txt')]
#print(docLabels)
#create a list data that stores the content of all text files in order of their names in docLabels
data = []
for doc in docLabels:
  data.append(open('inputfile/' + doc,encoding='utf-8').read())
#print('data')
#print(data)  


it = LabeledLineSentence(data, docLabels)
it.doc_list.sort
print('doc_list len:',len(it.doc_list))
print('labels_list len:',len(it.labels_list))
model = gensim.models.Doc2Vec(size=300, window=10, min_count=10, workers=11,alpha=0.025, min_alpha=0.025)
model.build_vocab(it)
#print('model vocabulary')
#print(model.wv.vocab)
#print()
for epoch in range(10):
    model.train(it,total_examples=model.corpus_count,epochs=model.iter)
    model.alpha -= 0.002            # decrease the learning rate
    model.min_alpha = model.alpha       # fix the learning rate, no deca
#    model.train(it)
model.save("doc2vec.model")
#loading the model
d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[1]
#print('vocabulary')
#print(d2v_model.wv.vocab)
#similar_doc = d2v_model.docvecs.most_similar(1)
sim_docs = {}
sim_words_dic = {}
for doc in docLabels:
  print("")
  print(doc)
  print("")
  docvec = d2v_model.docvecs[doc]
  #print(docvec)
  similar_doc = d2v_model.docvecs.most_similar(doc)
  sim_docs[doc] = similar_doc
  print(sim_docs[doc])
  print("")
  similar_words = d2v_model.wv.most_similar(positive=[docvec],topn=30)
  print(similar_words)
  sim_words_dic[doc] = similar_words


doc_list len: 24
labels_list len: 24

D3.3part0.txt

[('D3.5part0.txt', 0.40574997663497925), ('D5.2part6.txt', 0.3441813588142395), ('D5.2part3.txt', 0.3190234899520874), ('D5.2part5.txt', 0.2529369294643402), ('RMConceptDefinitonpart0.txt', 0.24621060490608215), ('D5.2part4.txt', 0.2038736343383789), ('D5.1part3.txt', 0.16562992334365845), ('D5.4part0.txt', 0.12707675993442535), ('D8.1part0.txt', 0.1262919008731842), ('D6.1part0.txt', 0.1217040866613388)]

[('eurosites', 0.4359957277774811), ('scatter', 0.41838839650154114), ('specialisation', 0.4183347225189209), ('incoherent', 0.4174373149871826), ('beam', 0.4105796813964844), ('opensearch', 0.3893756866455078), ('moist', 0.38170158863067627), ('conversion', 0.3738842010498047), ('visualisation', 0.370138555765152), ('ring', 0.3679256737232208), ('transmission', 0.3676825165748596), ('signal', 0.36604058742523193), ('gco', 0.3606894910335541), ('waveform', 0.3505513072013855), ('namespace', 0.34487345814704895), ('buffer', 0.343495

In [450]:
from IPython.core.display import display, HTML
from string import Template
import json, random
import json

In [451]:
#HTML engine that loads D3 remotely
HTML('<script src="lib/d3/d3.v3.min.js"></script>')

In [452]:
css_text = '''
* {
    box-sizing: border-box;
}

/* Create two unequal columns that floats next to each other */
.column {
    float: left;
    padding: 10px;
    /*height: 300px; *//* Should be removed. Only for demonstration */
}

.left {
  width: 50%;
}

.right {
  width: 50%;
}

/* Clear floats after the columns */
.row:after {
    content: "";
    display: table;
    clear: both;
}

'''

In [507]:
js_text_template = Template('''
var doclabels = $JdocLabels ;
var Similar_docs =  $similar_docs;
var Similar_Words = $Jsim_words_dic;

var docDiv = document.getElementById("docDiv");

for (var i = 0; i < doclabels.length; i++) {
    var checkBox = document.createElement("input");
    var label = document.createElement("label");
    checkBox.type = "checkbox";
    checkBox.value = doclabels[i];
    docDiv.appendChild(checkBox);
    docDiv.appendChild(label);
    label.appendChild(document.createTextNode(doclabels[i]));
}


function myFunction() 
{

var docDiv = document.getElementById("docDiv");
count = 0;
for (var i = 0; i < docDiv.childNodes.length; i++) 
{
var e = docDiv.childNodes[i];
if (e.checked == true)
count ++;
}
if (count != 1) 
 {
 document.getElementById("order").value = "Please select only one document";
 

 for (var i = 0; i < docDiv.childNodes.length; i++) 
     {
      var e = docDiv.childNodes[i];
      e.checked = false;
      }
 }
else
 {
     var checkedValue = null; 
     var docDiv = document.getElementById("docDiv");
    for (var i = 0; i < docDiv.childNodes.length; i++) 
    {
        var e = docDiv.childNodes[i];
          if(e.checked){
               checkedValue = e.value;
               document.getElementById("order").value = "you selected: " + checkedValue;
               break;
          }
    }
        var var_name = "docselected";
        var command = var_name + " = '" + checkedValue + "'";
        var kernel = IPython.notebook.kernel;
        kernel.execute(command);

        document.getElementById("pridoc").value = checkedValue;
        readTextFile("inputfile/"+checkedValue,"graph-div")
/*   
 var value;
for (var key in Similar_docs) {
    value = Similar_docs[key];
    alert(key +" : " +value);
    var top = value[0];
    alert(top);
    // your code here...
}
*/
        //get similar docs
        var sim = Similar_docs[checkedValue];
        //alert(sim);
        //alert(sim[0]);
        //alert(typeof(sim[0]))
        var top0 = String(sim[0]);
        //alert(top0);
        var topdoc = top0.split(",");
        var filename = topdoc[0];
        
        //alert(filename);
        document.getElementById("simdoc").value = filename;
        
        readTextFile("inputfile/"+filename,"graph-div2")
        
        //get similar words
        var words = [];
        var sim2 = Similar_Words[checkedValue];
        //alert(sim2);
        //alert(sim2[0]);
        //alert(sim2[1]);
        //alert(sim2[2]);
        //alert(sim2.length);
        for (var i =0; i < sim2.length; i++) 
        {
            //alert(sim2[i]);
            pairs = String(sim2[i]).split(",");
            //alert(pairs);
            //alert(pairs[0]);
            //alert(pairs[1]);
            words.push(pairs[0]);
        }
        markText(words);
        
 }//end else
    
}

function markText(words)
{
     var leftText = document.getElementById("graph-div").innerHTML;
     var newleftText = leftText;
     var rightText =  document.getElementById("graph-div2").innerHTML;
     var newrightText = rightText;
     for (var i =0; i < words.length; i++) 
        {
        leftText = newleftText;
        rightText = newrightText;
        console.log(words[i]);
        replace = '<font color="yellow">' + words[i] + '</font>';
        console.log(replace);
        newleftText = leftText.replace(new RegExp(words[i], 'g'), replace);
        newrightText = rightText.replace(new RegExp(words[i], 'g'), replace);
        }
        console.log(newleftText);
        console.log(newrightText);
        document.getElementById("graph-div").innerHTML = newleftText;
        document.getElementById("graph-div2").innerHTML = newrightText;
}
 
function readTextFile(file,id)
{
    var rawFile = new XMLHttpRequest();
    rawFile.open("GET", file, false);
    rawFile.onreadystatechange = function ()
    {
        if(rawFile.readyState === 4)
        {
            if(rawFile.status === 200 || rawFile.status == 0)
            {
                var allText = rawFile.responseText;
                var obj = document.getElementById(id); 
                obj.innerHTML = ""
                obj.innerHTML = allText; 
            }
        }
    }
    rawFile.send(null);
}
''')


In [508]:
html_template = Template(
'''
<style> $css_text </style>
<form name="docboxes">
<div id="docDiv">     </div>
<br>
<input type="button" onclick="myFunction()" value="Submit">

<br>
<input type="text" id="order" size="50">



<br>

<br>

<div class="row">
<div><input type="text" id="pridoc"  size="50" readonly></div>
  <div class="column left" id="graph-div" style="background-color:#aaa;">
  </div>
  
<div><input type="text" id="simdoc"  size="50" readonly></div>
  <div class="column right" id="graph-div2" style="background-color:#bbb;">
</div>
</div>
</form>
<script> $js_text </script>
''')
js_text = js_text_template.substitute({'Jsim_words_dic':json.dumps(sim_words_dic),'similar_docs':json.dumps(sim_docs) ,'JdocLabels':json.dumps(docLabels),'graphdiv':'graph-div','graphdiv2':'graph-div2'})
#print(json.dumps(sim_docs))
#similardocs = sim_docs[docselected]
#topSimilardoc = sim_docs[docselected][0][0]
#js_text = js_text_template.substitute({'keyDoc':keyDocContent})

HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))



